In [1]:
import networkx as nx
import math
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from networkx.algorithms import bipartite
import sys
import contextlib
import networkx as nx
import pandas as pd
from ipynb.fs.defs.functions import *

## Simulation of empirical networks
This notebook outlines how to numerically simulate the fixation probability of a weighted empirical network. We demonstrate the process using the example of the ants colony weighted networks. The other empirical networks were similarly evaluated.

In [2]:


# Load the DataFrame
ants = pd.read_csv("insecta-ant-trophallaxis-colony2.csv")

# Create an undirected graph
G = nx.Graph()

# Iterate through the DataFrame to update the network
for index, row in ants.iterrows():
    node1 = int(row['n1'])
    node2 = int(row['n2'])
    
    # Check if the edge already exists
    if G.has_edge(node1, node2):
        # If the edge exists, increment the weight by one
        G[node1][node2]['weight'] += 1
    else:
        # If the edge does not exist, add it with a weight of one
        G.add_edge(node1, node2, weight=1)

# Now G contains the undirected weighted network
mapping = {39:0}
Gprime = nx.relabel_nodes(G, mapping)

In [ ]:
#Simulating the empirical network of ants
import time
from timeit import default_timer as timer
from multiprocessing import Pool, cpu_count

def main(j):
    
    start = timer()
    arg=[]
    c=[]

    print(f'starting computations on {cpu_count()} cores')
    
    for i in range(0,cpu_count()):
        arg.append((Gprime,350000,j))


    with Pool() as pool:
        c.append(pool.starmap(weighted_simulation_count, arg))
        

    end = timer()
    print(f'elapsed time: {end - start}')
    return (sum(c[0])/len(c[0]))

if __name__ == '__main__':
    c=[]
    d_weighted_ants_39_nodes_new=[]
    for k in range(7,17):
        d_weighted_ants_39_nodes_new.append(main(k/10))
        np.save('d_weighted_ants_39_nodes_new.npy' , np.array(d_weighted_ants_39_nodes_new))

starting computations on 40 cores
